# Tutorial 3
In this tutorial, we are going to directly train a simple SNN with a single hidden layer using EventProp on the MNIST dataset, converted to a latency spike code.

Clearly, this is far from a state of the art architecture, but it still achieves 94% accuracy on MNIST.
## Install PyGeNN wheel from Google Drive
Download wheel file

In [1]:
!gdown 1fllqUtL_1_tyGzjNHSR-9i5fXFI9jqAi 

Downloading...
From: https://drive.google.com/uc?id=1fllqUtL_1_tyGzjNHSR-9i5fXFI9jqAi
To: /content/pygenn-4.8.1-cp310-cp310-linux_x86_64.whl
100% 22.3M/22.3M [00:00<00:00, 38.3MB/s]


and then install PyGeNN from wheel file

In [2]:
!pip install pygenn-4.8.1-cp310-cp310-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./pygenn-4.8.1-cp310-cp310-linux_x86_64.whl


and checkout mlGeNN from git and install

In [3]:
!git clone --branch master https://github.com/genn-team/ml_genn.git
!pip install ml_genn/ml_genn

Cloning into 'ml_genn'...
remote: Enumerating objects: 6759, done.
remote: Counting objects: 100% (2576/2576), done.
remote: Compressing objects: 100% (970/970), done.
remote: Total 6759 (delta 1604), reused 2479 (delta 1579), pack-reused 4183
Receiving objects: 100% (6759/6759), 37.13 MiB | 12.76 MiB/s, done.
Resolving deltas: 100% (4457/4457), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./ml_genn/ml_genn
  Preparing metadata (setup.py) ... done
  Created wheel for ml-genn: filename=ml_genn-2.0.0-py3-none-any.whl size=98340 sha256=2b9c2f27b7af1e1dc1d3f3086bf456d5d90b200db931a18f4b8f04536d2076c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-3002y2lh/wheels/3f/cf/27/0e9dec4bb1be2afac4b38c2dfb4ce0bc164ce1ecb32b6f91b8
Successfully built ml-genn


Set environment variable to allow GeNN to find CUDA

In [4]:
%env CUDA_PATH=/usr/local/cuda

env: CUDA_PATH=/usr/local/cuda


## Install MNIST package

In [5]:
!pip install mnist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Build model
Import standard modules and required mlGeNN classes

In [11]:
import mnist
import numpy as np
import matplotlib.pyplot as plt

from ml_genn import InputLayer, Layer, SequentialNetwork
from ml_genn.callbacks import Checkpoint
from ml_genn.compilers import EventPropCompiler, InferenceCompiler
from ml_genn.connectivity import Dense
from ml_genn.initializers import Normal
from ml_genn.neurons import LeakyIntegrate, LeakyIntegrateFire, SpikeInput
from ml_genn.optimisers import Adam
from ml_genn.serialisers import Numpy
from ml_genn.synapses import Exponential

from ml_genn.utils.data import (calc_latest_spike_time, linear_latency_encode_data)

from ml_genn.compilers.event_prop_compiler import default_params


##Parameters

Define some model parameters


In [7]:
NUM_INPUT = 28 * 28
NUM_HIDDEN = 128
NUM_OUTPUT = 10
BATCH_SIZE = 128

## Latency encoding
There are numerous ways to encode images using spikes but here we are going to emit a single spike for each spike at a time proportional the each pixel's intensity$.

In [8]:
train_spikes = linear_latency_encode_data(mnist.train_images(), 20.0)

## Network definition
Because our network is entirely feedforward, we can define it as a ``SequentialNetwork`` where each layer is automatically connected to the previous layer. As we have converted the MNIST dataset to spikes, we will use a ``SpikeInput`` to inject these directly into the network. For our hidden layer we are going use standard Leaky integrate-and-fire neurons. Finally, we are going to use a non-spiking output layer and read classifications out of this by averaging it's membrane voltage.



In [9]:
# Create sequential model
serialiser = Numpy("latency_mnist_checkpoints")
network = SequentialNetwork(default_params)
with network:
    # Populations
    input = InputLayer(SpikeInput(max_spikes=BATCH_SIZE * NUM_INPUT),
                                  NUM_INPUT)
    hidden = Layer(Dense(Normal(mean=0.078, sd=0.045)), 
                   LeakyIntegrateFire(v_thresh=0.61, tau_mem=20.0,
                                      tau_refrac=5.0),
                   NUM_HIDDEN, Exponential(5.0))
    output = Layer(Dense(Normal(mean=0.2, sd=0.37)),
                   LeakyIntegrate(tau_mem=20.0, readout="avg_var"),
                   NUM_OUTPUT, Exponential(5.0))

## Compilation
In mlGeNN, in order to turn an abstract network description into something that can actually be used for training or inference you use a *compiler* class. Here, we use the ``EventPropCompiler`` to train with EventProp and specify batch size and how many timesteps to evaluate each example for as well as choosing our optimiser and loss function. Because this is a classification task, we want to use cross-entropy loss and, because our labels are specified in this way (rather than e.g. one-hot encoded), we use the sparse catgorical variant.

In [12]:
max_example_timesteps = int(np.ceil(calc_latest_spike_time(train_spikes)))
compiler = EventPropCompiler(example_timesteps=max_example_timesteps,
                         losses="sparse_categorical_crossentropy",
                         optimiser=Adam(1e-2), batch_size=BATCH_SIZE)
compiled_net = compiler.compile(network)

## Training
Now we will train the model for 10 epochs using our compiled network. To verify it's performance we take 10% of the training data as a validation split and add an additional callback to checkpoint weights every epoch.



In [13]:
with compiled_net:
    # Evaluate model on numpy dataset
    callbacks = ["batch_progress_bar", Checkpoint(serialiser)]
    compiled_net.train({input: train_spikes},
                       {output: mnist.train_labels()},
                       num_epochs=15, shuffle=True,
                       validation_split=0.1,
                       callbacks=callbacks)

  0%|          | 0/422 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

## Evaluate
Load weights checkpointed from last epoch:

In [ ]:
network.load((9,), serialiser)

Create an ``InferenceCompiler`` and compile network for inference:

In [ ]:
compiler = InferenceCompiler(evaluate_timesteps=max_example_timesteps,
                             batch_size=BATCH_SIZE)
compiled_net = compiler.compile(network)

Encode *test set* using the same log-latency encoding and evaluate it:

In [ ]:
test_spikes = log_latency_encode_data(mnist.test_images(), 20.0, 51)
with compiled_net:
    compiled_net.evaluate({input: test_spikes},
                          {output: mnist.test_labels()})

  0%|          | 0/79 [00:00<?, ?it/s]